# The Brute Force Paradox

Copyright Johns Hopkins University.  Not for distribution online or by any other means.

Adapted from a submission from a student in Foundations of Algorithms, 605.621.

## Closest Points

Given the task of finding the closest pair of points in a two-dimensional plane, students were tasked with implementing a brute force algorithm, but making at least one change to improve its runtime down from $\Theta(n^2)$.  Usually, brute force iterates over all pairs of points by way of a doubly nested ``for`` loop.  They were also asked to implement the closest points divide-and-conquer approach in the CLRS textbook that has a runtime of $O(n\log n)$.

To improve the brute force method, one clever student had the idea to sort the list of points beforehand by x value.  When processing point i, and in the loop that pairs i with all other points j, if the x difference between i and j is greater than the current minimum distance, then stop processing j and subsequent points.  Instead, *skip ahead* to the next i.  In this way, since all points following j have a greater x value, there is no need to consider them.

Students were directed to do testing against random points in a list of increasing size, from 3 to N points, and plotting/comparing brute force skip-ahead performance against divide-and-conquer.

## The Paradox
The submission in question did such a comparison, but the plot showed very similar asymptotic performance to CLRS.  It seemed that no matter how the points were randomized, the skip-ahead program always performed about the same as the divide-and-conquer.  This seemed crazy; we are always taught that $n\log n$ beats $n^2$, so why is the plot telling us otherwise?  Also, the plot seemed unstable - repeated trials showed near equal performance to divide-and-conquer, but sometimes the performance spiked a little bit high.  What is going on here? 

# Resolving the Paradox
It turns out that there is a reason for this happening.  The goal of this notebook is to help explore the nature of that phenomena.

# Instructions to Students
This is **not** an assignment that you need to turn in.  Instead, it is a tool to explore the phenomena, and help you with a homework problem.  The brute force implementation is given; if you prefer, you may paste in your CLRS implementation to experiment on similar sets of points, but you don't need to.

# Closest Pairs
Below is the implementation of the improved brute force algorithm, along with some instructor data structures and functions.



## Instructor Data Structures and Helpers

These are data structures used in this notebook.

In [ ]:
############################################################################
# INSTRUCTOR DATA STRUCTURES
# If you change these, don't break the instructor test block.
############################################################################
# Decorator example for counting function calls, adapted from 
# https://stackoverflow.com/a/21717396/8542716
def call_counter(f):
    """
    Adds a ".calls" variable to the function that increments w/every call.
    Set it to zero between iterations.  
    Example:
        @call_counter
        def sumallbelow(x):
          if x<1:
            return 0
          return x + sumallbelow(x-1)
        
        ...
        sumallbelow.calls = 0 # reset before any use
        print(f"The sum up to 4 is: {sumallbelow(4)}")
        print(f"and the call counts are {sumallbelow.calls}")

    """
    def wrapped(*args, **kwargs): # deal with any/all arguments
        wrapped.calls += 1
        return f(*args, **kwargs) # call the real function here
    wrapped.calls = 0
    return wrapped

class debugprint(object):
  """
  A debug print decorator useful for tracing the flow of recursive functions.
  It adds a "debugprint" method to each function call, that increases the 
  number of spaces on each call before printing the output.

  To use, add @debugprint on top of your function.  Then, within your function
  foo, put foo.debugprint("whatever"), and it will print "whatever" with 
  several spaces in front of it.  Recursive calls add to the spaces.  Enable
  with "debugprint.enabled = True", and disable w/False.

  There's also a counter called ".calls," which you reset like the counter
  decorator above.  Note, enabling/disabling does not affect the counter; the
  counter is always counting.

  Example:
      @debugprint
      def fib(N):
        fib.debugprint(f"inside fib({N})")
        if N<2:
          fib.debugprint(f"returning {N}")
          return N
        fib.debugprint(f"returning fib({N-1}) + fib({N-2})")
        return fib(N-1) + fib(N-2)

      # toggle this on/off as needed
      fib.enabled, fib.calls = True, 0
      print(f"The 3rd Fibonacci number is {fib(3)}")
      print(f"Number of calls is {fib.calls}")

  Output:
      inside fib(3)
      returning fib(2) + fib(1)
        inside fib(2)
        returning fib(1) + fib(0)
          inside fib(1)
          returning 1
          inside fib(0)
          returning 0
        inside fib(1)
        returning 1
      The 3rd Fibonacci number is 2
      Number of calls is 5

  Adapted from https://stackoverflow.com/a/862915/
  Also adding a calls counter because I cannot combine decorators
  """
  calls = 0
  enabled = False
  space = ""
  def __init__(self, func):
    self.func = func
  def __call__(self, *args, **kwargs):
    if self.enabled:
      # print 'Entering', self.func.func_name 
      # print '    args:', args, kwargs
      pass
    self.space += "  "
    self.calls += 1
    result = self.func(*args, **kwargs)
    self.space = self.space[:-2]
    return result
  def debugprint(self, *args, **kwargs):
    if self.enabled:
      print(self.space+" ".join(map(str,args)), **kwargs)

class Point(object):
    """
    Data structure for points.  Methods include P.x, P.y and other Pythonisms
    necessary to use with sets, tuples, etc.
    """
    def __init__(self, x, y): 
        self.x = x 
        self.y = y
    def __repr__(self):
        return f"Point({self.x},{self.y})" 
    def __eq__(self, other): # need both eq and hash if you want to use sets
        return self.x == other.x and self.y == other.y
    def __hash__(self):
        return hash((self.x, self.y))
    def __iter__(self): # required to convert to tuple
        for i in [self.x, self.y]:
          yield i

: 

## Brute Force Improved
Here is the implementation in question.  Notice that the call counts are on the distance function itself; to be fair, we must also include the calls to the sort function, which is captured through an external key function to keep track of comparison operations.

Also, if called w/debugging=True, it prints ``breaking`` whenever it decides to skip the remaining points j, or ``computing`` if computing the distance.  Use this sparingly, but you should see a mix of both in a random run.






In [ ]:
from math import sqrt

############################################################################
# STUDENT IMPLEMENTATION BLOCK - Brute Force Improved
############################################################################
# Student Euclidean Distance Function
@call_counter  # adds calls attribute to functions
def dist(p1, p2):
    """
    Return the Euclidean distance between the two given points

    p1, p2 are Points
    """
    d_x = p1.x - p2.x
    d_y = p1.y - p2.y
    d = sqrt(d_x**2 + d_y**2)
    return d

@call_counter
def mysortkey(p):
    """
    Purpose of this is to trip a call counter every time Python's sort makes
    a comparison.
    """
    return p.x

# Implementation of Student Closest Points algorithm
def closestPairBrute(points, debug=False):  # Preserve the function name and arguments
    """
    This is the Brute Force Improved n^2 implementation. This implementation first sorts
    the given list of points using Python's sort, then uses horizontal distance to determine
    whether the distance function needs to be called on a pair of points. There are two 
    improvements to this algorithm compared to the brute force algorithm:
    
        1. The horizontal distance is first computed to see if the distance function
           even needs to be called. Since the horizontal distance and vertical distance
           form a triangle with the Euclidean distance between two points, we can use
           the property that the hypotenuse of a triangle is always longer than either 
           leg of the triangle. 
        2. The points are sorted by x-value before finding the closest pair of points. 
           Since we are using horizontal distance to determine whether to call the 
           distance function, once we find that the horizontal distance exceeds the 
           current minimum distance, we no longer need to compare the first of these
           points with any other points later in the list (points with larger x-values).
    
    Return the two closest points, the Euclidean distance between them, and the number
    of times the dist() function was called.
    
    points is an array of Points
    """    
    # Sort the list of points by x-value
    points.sort(key=lambda pt: pt.x)
    mysortkey.calls = 0
    points.sort(key=mysortkey) # to count the steps of the sort function

    # Find the closest points
    dist.calls = 0
    # Maintain dist as part of a tuple w/points that make that distance
    min_tuple = (None, None, float('inf')) # p1, p2, and dist(p1, p2)
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            min_dist = min_tuple[2]
            # If the horizontal distance between point[i] and point[j] is 
            # greater than min_dist, move on to the next value of i
            if points[j].x - points[i].x >= min_dist:
                if debug: print("breaking")
                break
            else:
              if debug: print("computing distance")
              cur_tuple = (points[i], points[j], dist(points[i], points[j]))
              min_tuple = min(min_tuple, cur_tuple, key=lambda t: t[2]) # min of dist
    p1, p2, min_dist = min_tuple
    return p1, p2, min_dist, dist.calls + mysortkey.calls

: 

## CLRS Algorithm

This is for the closest pair of points described in the CLRS textbook, 3rd ed, in section 33.4.  Input and output will be as with the brute force method, but runs in $n\log(n)$ time.

Feel free to paste your own CLRS implementation in here.  Maintain the supplied function signatures, to remain compatible with test code further on.  You don't need to, but in case you want to experiment with different points, please do so.

In [ ]:
# STUDENT RECURSIVE FUNCTION GOES HERE

def closestPairRecursive(X, Y):
    """
    Returns the closest pairs of Points in X, the distance between these two
    points, and the number of calls to dist(). This function implements the 
    divide/conquer/combine algorithm described in CLRS, where we recurse on 
    the left and right half of X, then consider a 2-delta width from the 
    center in order to find the closest Euclidean pair.
    
    X is an array of Points, sorted by x-value
    Y is the same array of Points, but sorted on y-value
    """
    return Point(0,0), Point(0,0), 0, 0

: 

In [ ]:
############################################################################
# STUDENT IMPLEMENTATION BLOCK - CLRS
############################################################################
# Implementation of Student Closest Points algorithm
def closestPairCLRS(points):  # Preserve the function name and arguments
    """
    This is the kickoff function to the recursive function explained in CLRS.
    It does the sorting of the given Points prior to calling the recursive
    function, and returns the final output of the recursive function: the 
    two closest points, distance between them, and number of calls to dist()
    
    points is an array of Points
    """
    # Sort the array of points by x-value, and separately by y-value
    X = sorted(points, key=lambda pt: pt.x)
    Y = sorted(points, key=lambda pt: pt.y)
    
    dist.calls = 0
    return closestPairRecursive(X, Y)

: 

## Experiment Area
Here is where experiments can be run.

### Data Collection
This runs repeated tests over input sizes N to collect data for the plot section.

In [ ]:
from os import setuid
from random import randint

def getRuntime(func, inp):
  """
  Return the number of calls to dist() made to find the closest pair of 
  points using the given function and input array.
  
  func is the function to call (either closestPairBrute or closestPairCLRS)
  inp is array of points
  """
  try:
      dist.calls=0
      tup = func(inp) # p1, p2, dist, calls
      p1, p2, d, dc = tup
      return dc
  except Exception as e:
      print(" ")
      print("======== EXCEPTION ======== EXCEPTION ======== EXCEPTION")
      print("========",e)

        
def generateTestData(n, max_coord, worst=False):
    """
    Update to repeat trials "worst" times to find something bad for brute force,
    then get the counts based on that.
    """
    mybrutecounts = []
    myCLRScounts = []
        
    for input_len in range(n):
      mbf = (0,[])
      rand_points = []
      if worst:
        pass
      else:
        # random set of points (no duplicates)
        rset = set()
        while len(rset) < input_len: 
            x, y = randint(0,max_coord), randint(0,max_coord)
            rset.add(Point(x, y))
        rand_points = list(rset)
      # Run both functions on the randomly-generated input to get the number of calls to dist()
      bftot = getRuntime(closestPairBrute, rand_points)
      mbf = max(mbf, (bftot, rand_points), key=lambda t: t[0])
      rand_points = mbf[1]
      if input_len==6: # just a spot-check
          rand_points.sort(key=lambda p: p.x)
          print(rand_points)
          print([d2.x-d1.x for d1, d2 in zip(rand_points, rand_points[1:])])
          _ = closestPairBrute(rand_points, True) # just to print it out
      mybrutecounts.append(getRuntime(closestPairBrute, rand_points))
      myCLRScounts.append(getRuntime(closestPairCLRS, rand_points))

    return mybrutecounts, myCLRScounts

# Just run this to generate the unfiltered output.  Then, if you want, 
# implement your worst case guess under "if worst:" in the code block above,
# put in code to simulate the worst case for the algorithm, then change
# False to True down here:
mybrutecounts, myCLRScounts = generateTestData(200, 1000, False)
print(mybrutecounts[:10])
print(myCLRScounts[:10])


: 

### Plot
Here is the plot.

In [ ]:
# Code adapted from Sahil Sharma, used with permission, from Spring, 2021

from math import pow
from math import log2
from math import factorial

# REQUIRES mybrutecounts, myCLRScounts set above
N=len(mybrutecounts)

# benchmark data set generation
yTestVals1 = [0,] # linear for O(n)
yTestVals2 = [0,] # 1/3*n*log(n) for O(n/3*log(n))
yTestVals3 = [0,] # 2/3*n*log(n) for O(2n/3*log(n))
yTestVals4 = [0,] # n*log(n) for O(nlog(n))
yTestVals5 = [0,] # quadratic for O(n^2)

# generate bench mark data for O(n), O(3n/2), O(n/2*log(n)), O(nlog(n)), and O(n^2)
for i in range(2, N+1):
    # UPDATE THE COEFFICIENTS TO THESE FUNCTIONS TO BRING SOME IN "TIGHT" TO
    # THE BRUTE FORCE, and some to the CLRS, or add others.
    yTestVals1.append(i)
    yTestVals2.append(0.3*i*log2(i))  # change this benchmark
    yTestVals3.append(2*i/3*log2(i))
    yTestVals4.append(i*log2(i))
    yTestVals5.append(pow(i, 2))

# %matplotlib inline
import matplotlib.pyplot as plt

# Brute vs. CLRS execution steps
plt.rcParams['figure.figsize'] = [10,5]
fig, (ax1) = plt.subplots(1, 1)
fig.suptitle('Runtime Analysis', size="xx-large")
fig.patch.set_facecolor('xkcd:white')
# plotting nearest neighbor execution steps
ax1.set_title('Brute Improved vs. CLRS', size="x-large")
ax1.plot(range(N), mybrutecounts, "rx-", markersize=10, linewidth=4, label="Skip-Ahead Brute Counts")
if myCLRScounts[-1]:
  ax1.plot(range(N), myCLRScounts, "gx-", markersize=10, linewidth=4, label="my CLRS Counts")
ax1.plot(range(N), yTestVals2, "y--", markersize=0, linewidth=5, label="typical CLRS counts")
ax1.plot(range(N), yTestVals1, "k^-", markersize=0, linewidth=1, label="benchmark O(n)")
ax1.plot(range(N), yTestVals3, "m^-", markersize=0, linewidth=1, label="benchmark O(2n/3*log(n))")
# ax1.plot(range(N), yTestVals4, "y^-", markersize=0, linewidth=1, label="benchmark O(nlog(n))")
ax1.plot(range(N), yTestVals5, "b^-", markersize=0, linewidth=1, label="benchmark O(n^2)")
ax1.grid(b=True, which='major', axis='both')
ax1.set(xlabel = '# points in input array', ylabel = '# calls to dist()')
ax1.legend(loc=(1.05,0.60), scatterpoints=1)
ax1.tick_params(axis="both", which="major", labelsize=14)
ax1.axis([0,N,0,0.5*N*log2(N)]) 

: 

### Challenge Questions
Why does the modified brute force appear to be as efficient as the known-optimal CLRS implementation?  Could it be that simply sorting the points frees us from the pesky $\Theta(n^2)$ bound?  Or is this an exaggerated stretch of the "O" bound in the $O(n^2)$ asymptotic relation?


Can you find an input that "breaks" it and brings it closer to its expected $n^2$ bound?  What is the general pattern of a "bad" input to the algorithm?  If so, would that same input affect CLRS as well?

Finally, what are the tradeoffs in using the skip-ahead brute force optimization, vs. the CLRS code?  Think of one reason to use skip-ahead over CLRS.  Hint, one reason has nothing to do with asymptotic behavior.